In [1]:
%load_ext autoreload
%autoreload 2

# Librerías

In [2]:
import numpy as np
import timber_nds
from timber_nds import settings, calculation
import timber_nds.settings as settings
import timber_nds.calculation as calculation
import timber_nds.design as design


# Parámetros


A continuación, se definen todos los parámetros necesarios para el cuaderno, incluyendo factores de reducción de resistencia, propiedades de materiales y dimensiones geométricas.

**Factores de Reducción de Resistencia:**

*   `phi_tension`: Factor de reducción de resistencia para tensión.
*   `phi_bending`: Factor de reducción de resistencia para flexión.
*   `phi_shear`: Factor de reducción de resistencia para corte.
*   `phi_compression`: Factor de reducción de resistencia para compresión.
*    `phi_perp_compression`: Factor de reducción de resistencia para compresión perpendicular a la fibra.

**Propiedades del Material:**

*   `species`: Especie de madera.
*   `specific_gravity`: Gravedad específica de la madera.
*   `fibre_saturation_point`: Punto de saturación de la fibra de la madera (%).
*   `moisture_content`: Contenido de humedad de la madera (%).
*   `e_modulus`: Módulo de elasticidad de la madera (Pa).
*  `fb`: Resistencia a la flexión (Pa).
* `ft`: Resistencia a la tracción (Pa).
* `fv`: Resistencia al corte (Pa).
* `fc`: Resistencia a la compresión paralela a la fibra (Pa).
* `fc_perp`: Resistencia a la compresión perpendicular a la fibra (Pa).

**Dimensiones Geométricas:**

*   `width`: Ancho de la sección transversal (m).
*   `depth`: Profundidad de la sección transversal (m).
*   `length`: Longitud del elemento (m).
*   `k_factor`: Factor de longitud efectiva.

**Factores de Ajuste:**

*   Se definen varios factores de ajuste para tensión, flexión, corte, compresión, compresión perpendicular a la fibra y módulo de elasticidad.


In [3]:
# Factors
phi_tension = 0.90
phi_bending = 0.85
phi_shear = 0.75
phi_compression = 0.90
phi_perp_compression = 0.65

# Material Properties
species = "Douglas Fir-Larch"
specific_gravity = 0.55
fibre_saturation_point = 28
moisture_content = 19 #19%
e_modulus = 115000 # kgf/cm2
fb = 280  # kgf/cm2
ft = 280  # kgf/cm2a
fv = 45  # kgf/cm2
fc = 280  # kgf/cm2
fc_perp = 28  # kgf/cm2


# Geometry
width = 5. # cm
depth = 15. # cm
length = 400 # cm
k_factor = 1.0

# Adjustment factors initialization
ft_factors = settings.TensionAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.85,
    due_temperature=1.0,
    due_size=1.0,
    due_incising=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fb_factors_yy = settings.BendingAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.85,
    due_temperature=1.0,
    due_size=1.0,
    due_beam_stability=1.0,
    due_flat_use=1.0,
    due_incising=1.0,
    due_repetitive_member=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fb_factors_zz = settings.BendingAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.85,
    due_temperature=1.0,
    due_size=1.0,
    due_beam_stability=1.0,
    due_flat_use=1.0,
    due_incising=1.0,
    due_repetitive_member=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fv_factors = settings.ShearAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.97,
    due_temperature=1.0,
    due_incising=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fc_factors_yy = settings.CompressionAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.80,
    due_temperature=1.0,
    due_size=1.0,
    due_incising=1.0,
    due_column_stability=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fc_factors_zz = settings.CompressionAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.80,
    due_temperature=1.0,
    due_size=1.0,
    due_incising=1.0,
    due_column_stability=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fc_perp_factors = settings.PerpendicularAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.67,
    due_temperature=1.0,
    due_beam_stability=1.0,
    due_size=1.0,
    due_flat_use=1.0,
    due_incising=1.0,
    due_repetitive_member=1.0,
    due_column_stability=1.0,
    due_buckling_stiffness=1.0,
    due_bearing_area=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

e_factors = settings.ElasticModulusAdjustmentFactors(
    due_moisture=0.90,
    due_temperature=1.0,
    due_incising=1.0,
    due_format_conversion=1.0
)

# Ejemplo


Este cuaderno demostrará el cálculo del peso, las propiedades geométricas y la capacidad de una viga de madera de sección rectangular. 
Comenzaremos calculando el peso de la viga, luego sus propiedades geométricas, y finalmente, integraremos todos los resultados para calcular la capacidad de la viga.
El ejemplo se desarrollará de manera continua a través de todas las secciones, ilustrando el flujo de trabajo típico en el diseño de estructuras de madera.


## Ejemplo de Función 1: Cálculo del Peso


Esta sección demuestra cómo calcular el peso de un elemento de madera utilizando la clase `WeightCalculator`.
Primero, se crea una instancia de `WoodMaterial`, `RectangularSection` y `MemberDefinition` con los parámetros definidos anteriormente.
Luego, se crea una instancia de `WeightCalculator` y se calcula el peso del elemento para el contenido de humedad especificado.


In [4]:
# Creating instances of classes
wood_material = settings.WoodMaterial(
    name=species,
    specific_gravity=specific_gravity,
    fibre_saturation_point=fibre_saturation_point,
    tension_strength=ft,
    bending_strength=fb,
    shear_strength=fv,
    compression_perpendicular_strength=fc_perp,
    compression_parallel_strength=fc,
    elastic_modulus=e_modulus,
    color="Brown"
)

section = settings.RectangularSection("sección 1", depth=depth, width=width)
element = settings.MemberDefinition("elemento 1", length=length, )
weight_calculator = calculation.WeightCalculator(wood_material, section, element)
material_density = weight_calculator.calculate_density_at_moisture_content(moisture_content)
element_weight = weight_calculator.calculate_weight_at_moisture_content(moisture_content)

print(f"La densidad de la madera es: {material_density:.2f} kgf/m³")
print(f"El peso del elemento de madera es: {element_weight:.2f} kgf")

densidad = 567.157712305026
dimensiones=5.0, 15.0 y 400
La densidad de la madera es: 567.16 kgf/m³
El peso del elemento de madera es: 17.01 kgf


In [5]:
depth/100 * width/100 * length / 100 * material_density

17.01473136915078

## Ejemplo de Función 2: Propiedades Geométricas


En esta sección, calculamos las propiedades geométricas de la sección rectangular utilizando la clase `RectangularSectionProperties`.
Se crea una instancia de `RectangularSectionProperties` con el ancho y la profundidad de la sección.
Luego, calculamos el área, los momentos de inercia, los módulos de sección y los radios de giro.


In [6]:

# Calculate Section Properties
section_properties = calculation.RectangularSectionProperties(width, depth)
area = section_properties.area()
i_yy = section_properties.moment_of_inertia("yy")
i_zz= section_properties.moment_of_inertia("zz")
elastic_section_modulus_yy = section_properties.elastic_section_modulus("yy")
elastic_section_modulus_zz = section_properties.elastic_section_modulus("zz")
radius_gyration_yy = section_properties.radius_of_gyration("yy")
radius_gyration_zz = section_properties.radius_of_gyration("zz")

print(f"Área de la sección: {area:.1f} cm^2")
print(f"Momento de inercia Iyy: {i_yy:.1f} cm^4")
print(f"Momento de inercia Izz: {i_zz:.1f} cm^4")
print(f"Módulo de sección elástico Sxx: {elastic_section_modulus_yy:.1f} cm^3")
print(f"Módulo de sección elástico Syy: {elastic_section_modulus_zz:.1f} cm^3")
print(f"Radio de giro r_yy: {radius_gyration_yy:.1f} cm")
print(f"Radio de giro r_zz: {radius_gyration_zz:.1f} cm")
    

Área de la sección: 75.0 cm^2
Momento de inercia Iyy: 1406.2 cm^4
Momento de inercia Izz: 156.2 cm^4
Módulo de sección elástico Sxx: 187.5 cm^3
Módulo de sección elástico Syy: 62.5 cm^3
Radio de giro r_yy: 4.3 cm
Radio de giro r_zz: 1.4 cm


## Ejemplo de Función 3: Longitud Efectiva


Aquí, calculamos la longitud efectiva de la viga usando la función `effective_length`.
La longitud efectiva se calcula multiplicando la longitud real de la viga por el factor de longitud efectiva (K).


In [7]:
# Calculate Effective Length
effective_length_value = calculation.effective_length(k_factor, length)
print(f"Longitud efectiva de la viga: {effective_length_value/100:.2f} m")
    

Longitud efectiva de la viga: 4.00 m


## Ejemplo de Función 4: Factores Combinados


En esta sección, demostramos cómo utilizar la clase `WoodElementCalculator` para calcular los factores combinados.
Se crea una instancia de `WoodElementCalculator` con todos los factores de ajuste definidos anteriormente.
Luego, se calcula el factor combinado para cada propiedad mecánica (tensión, flexión, corte, compresión, compresión perpendicular a la fibra y módulo de elasticidad).


In [8]:
# Crear una instancia de WoodElementCalculator
wood_element_calculator = design.WoodElementCalculator(
    tension_factors = ft_factors,
    bending_factors_yy = fb_factors_yy,
    bending_factors_zz = fb_factors_zz,
    shear_factors = fv_factors,
    compression_factors_yy = fc_factors_yy,
    compression_factors_zz = fc_factors_zz,
    compression_perp_factors = fc_perp_factors,
    elastic_modulus_factors = e_factors,
    material_properties = wood_material,
    section_properties = section_properties
)

# Calcular los factores combinados
combined_factors = wood_element_calculator.calculate_combined_factors()
print(f"Factores combinados: {combined_factors}")
    

Factores combinados: {'tension': np.float64(0.85), 'bending_yy': np.float64(0.85), 'bending_zz': np.float64(0.85), 'shear': np.float64(0.97), 'compression_yy': np.float64(0.8), 'compression_zz': np.float64(0.8), 'compression_perp': np.float64(0.67), 'elastic_modulus': np.float64(0.9)}


## Ejemplo de Función 5: Resistencia del elemento

In [9]:
tension_strength = wood_element_calculator.tension_strength()
bending_strength_yy = wood_element_calculator.bending_strength("yy")
bending_strength_zz = wood_element_calculator.bending_strength("zz")
shear_strength = wood_element_calculator.shear_strength()
compression_strength_yy = wood_element_calculator.compression_strength("yy")
compression_strength_zz = wood_element_calculator.compression_strength("zz")
compression_perp_strength = wood_element_calculator.compression_perp_strength()


In [10]:
print(f'La resistencia a la tensión es {float(tension_strength)} kgf')
print(f'La resistencia a la flexión es {float(bending_strength_yy) / 100} kgfm')
print(f'La resistencia a la flexión es {float(bending_strength_zz) / 100} kgfm')
print(f'La resistencia al cortante es {float(shear_strength)} kgf')
print(f'La resistencia a la compresión yy es {float(compression_strength_yy)} kgf')
print(f'La resistencia a la compresión zz es {float(compression_strength_zz)} kgf')
print(f'La resistencia a la compresión perpendicular es {float(compression_perp_strength)} kgf')

La resistencia a la tensión es 17850.0 kgf
La resistencia a la flexión es 446.25 kgfm
La resistencia a la flexión es 148.75 kgfm
La resistencia al cortante es 2182.5 kgf
La resistencia a la compresión yy es 16800.0 kgf
La resistencia a la compresión zz es 16800.0 kgf
La resistencia a la compresión perpendicular es 18.76 kgf


## Ejemplo de funciones para cálculo de todos los elementos

In [11]:
# Factors
phi_tension = 0.90
phi_bending = 0.85
phi_shear = 0.75
phi_compression = 0.90
phi_perp_compression = 0.65

# Material Properties
species = "Douglas Fir-Larch"
specific_gravity = 0.55
fibre_saturation_point = 28
moisture_content = 19 #19%
e_modulus = 115000 # kgf/cm2
fb = 280  # kgf/cm2
ft = 280  # kgf/cm2a
fv = 45  # kgf/cm2
fc = 280  # kgf/cm2
fc_perp = 28  # kgf/cm2


# Geometry
width = 5. # cm
depth = 15. # cm
length = 400 # cm
k_factor = 1.0

# Geometry
width_2 = 100. # cm
depth_2 = 150. # cm
length_2 = 200 # cm
k_factor_2 = 2.0

# Geometry
width_3 = 15. # cm
depth_3 = 15. # cm
length_3 = 200 # cm
k_factor_3 = 2.0

# Adjustment factors initialization
ft_factors = settings.TensionAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.85,
    due_temperature=1.0,
    due_size=1.0,
    due_incising=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fb_factors_yy = settings.BendingAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.85,
    due_temperature=1.0,
    due_size=1.0,
    due_beam_stability=1.0,
    due_flat_use=1.0,
    due_incising=1.0,
    due_repetitive_member=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fb_factors_zz = settings.BendingAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.85,
    due_temperature=1.0,
    due_size=1.0,
    due_beam_stability=1.0,
    due_flat_use=1.0,
    due_incising=1.0,
    due_repetitive_member=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fv_factors = settings.ShearAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.97,
    due_temperature=1.0,
    due_incising=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fc_factors_yy = settings.CompressionAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.80,
    due_temperature=1.0,
    due_size=1.0,
    due_incising=1.0,
    due_column_stability=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fc_factors_zz = settings.CompressionAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.80,
    due_temperature=1.0,
    due_size=1.0,
    due_incising=1.0,
    due_column_stability=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fc_perp_factors = settings.PerpendicularAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.67,
    due_temperature=1.0,
    due_beam_stability=1.0,
    due_size=1.0,
    due_flat_use=1.0,
    due_incising=1.0,
    due_repetitive_member=1.0,
    due_column_stability=1.0,
    due_buckling_stiffness=1.0,
    due_bearing_area=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

e_factors = settings.ElasticModulusAdjustmentFactors(
    due_moisture=0.90,
    due_temperature=1.0,
    due_incising=1.0,
    due_format_conversion=1.0
)

In [12]:
section = settings.RectangularSection("sección 1", depth=depth, width=width)
section_2 = settings.RectangularSection("sección 2", depth=depth_2, width=width_2)
section_3 = settings.RectangularSection("sección 3", depth=depth_3, width=width_3)
list_sections = [section, section_2, section_3]
list_sections

[RectangularSection(name='sección 1', depth=15.0, width=5.0),
 RectangularSection(name='sección 2', depth=150.0, width=100.0),
 RectangularSection(name='sección 3', depth=15.0, width=15.0)]

In [13]:
element = settings.MemberDefinition("elemento 1", length=length, )
element_2 = settings.MemberDefinition("elemento 2", length=length_2, )
element_3 = settings.MemberDefinition("elemento 3", length=length_3, )
list_members = [element, element_2, element_3]

In [14]:
list_members

[MemberDefinition(name='elemento 1', length=400, effective_length_factor_yy=1.0, effective_length_factor_zz=1.0),
 MemberDefinition(name='elemento 2', length=200, effective_length_factor_yy=1.0, effective_length_factor_zz=1.0),
 MemberDefinition(name='elemento 3', length=200, effective_length_factor_yy=1.0, effective_length_factor_zz=1.0)]

In [15]:
force = settings.Forces(
    name="force_1",
    axial_force=10000,
    shear_y= 10.0,
    shear_z= 10.0,
    moment_xx = 0.0,
    moment_yy = 0.0,
    moment_zz = 0.0
)

force_2 = settings.Forces(
    name="force_2",
    axial_force=0.0,
    shear_y= 00.0,
    shear_z= 00.0,
    moment_xx = 30.0,
    moment_yy = 400.0,
    moment_zz = 50.0
)
force_3 = settings.Forces(
    name="force_3",
    axial_force=-10000,
    shear_y= 10.0,
    shear_z= 0.0,
    moment_xx = 0.0,
    moment_yy = 1000.,
    moment_zz = 0.0
)

force_4 = settings.Forces(
    name="force_4",
    axial_force=-2000,
    shear_y= 10.0,
    shear_z= 20.0,
    moment_xx = 30.0,
    moment_yy = 400.0,
    moment_zz = 50.0
)


In [16]:
list_forces = [force, force_2, force_3, force_4]

In [17]:
design.calculate_dcr_for_wood_elements(
    element= element,
    section= section,
    forces=force,
    material= wood_material,
    tension_factors= ft_factors,
    bending_factors_yy= fb_factors_yy,
    bending_factors_zz= fb_factors_zz,
    shear_factors= fv_factors,
    compression_factors_yy= fc_factors_yy,
    compression_factors_zz= fc_factors_zz,
    compression_perp_factors= fc_perp_factors,
    elastic_modulus_factors=e_factors
)

{'axial tension': 10000,
 'axial compression': 0,
 'moment yy': 0.0,
 'moment zz': 0.0,
 'shear y': 10.0,
 'shear z': 10.0,
 'tension (dcr)': 0.5602240896358543,
 'biaxial bending (dcr)': 0.0,
 'bending and tension (dcr)': 0.5602240896358543,
 'compression (dcr)': 0.0,
 'bending and compression (dcr)': 0.0,
 'shear y (dcr)': 0.004581901489117984,
 'shear z (dcr)': 0.004581901489117984,
 'compression perpendicular': 10.0,
 'compression perpendicular (dcr)': 0.5330490405117271}

# Pruebas


### Prueba para `WeightCalculator.calculate_density_at_moisture_content`

```python



In [18]:
force_results = design.check_for_all_forces(
    section, element, force, wood_material, ft_factors,
    fb_factors_yy, fb_factors_zz,
    fv_factors,
    fc_factors_yy, fc_factors_zz, fc_perp_factors,
    e_factors
    )

Checking for all forces: 100%|██████████| 1/1 [00:00<00:00, 1000.07it/s]

Calculando para la fuerza: force_1


In [19]:
force_results

,member,section,force,dcr_max,axial tension,axial compression,moment yy,moment zz,shear y,shear z,tension (dcr),biaxial bending (dcr),bending and tension (dcr),compression (dcr),bending and compression (dcr),shear y (dcr),shear z (dcr),compression perpendicular,compression perpendicular (dcr)
0,elemento 1,sección 1,force_1,0.004582,10000,0,0.0,0.0,10.0,10.0,0.560224,0.0,0.560224,0.0,0.0,0.004582,0.004582,10.0,0.533049


In [20]:
forces_results = design.check_for_all_forces(
    section, element, list_forces, wood_material, ft_factors,
    fb_factors_yy, fb_factors_zz,
    fv_factors,
    fc_factors_yy, fc_factors_zz, fc_perp_factors,
    e_factors
    )

Checking for all forces: 100%|██████████| 4/4 [00:00<?, ?it/s]

Calculando para la fuerza: force_1
Calculando para la fuerza: force_2
Calculando para la fuerza: force_3
Calculando para la fuerza: force_4


In [21]:
forces_results

,member,section,force,dcr_max,axial tension,axial compression,moment yy,moment zz,shear y,shear z,tension (dcr),biaxial bending (dcr),bending and tension (dcr),compression (dcr),bending and compression (dcr),shear y (dcr),shear z (dcr),compression perpendicular,compression perpendicular (dcr)
0,elemento 1,sección 1,force_1,0.004582,10000,0.0,0.0,0.0,10.0,10.0,0.560224,0.000000,0.560224,0.000000,0.000000,0.004582,0.004582,10.0,0.533049
1,elemento 1,sección 1,force_2,0.012325,0,0.0,400.0,50.0,0.0,0.0,0.000000,0.012325,0.012325,0.000000,0.012325,0.000000,0.000000,0.0,0.000000
2,elemento 1,sección 1,force_3,0.595238,0,10000.0,1000.0,0.0,10.0,0.0,0.000000,0.022409,0.022409,0.595238,0.376717,0.004582,0.000000,0.0,0.000000
3,elemento 1,sección 1,force_4,0.119048,0,2000.0,400.0,50.0,10.0,20.0,0.000000,0.012325,0.012325,0.119048,0.026497,0.004582,0.009164,20.0,1.066098


In [22]:
section_result = design.check_for_all_sections(
    section_2, element, force, wood_material, ft_factors,
    fb_factors_yy, fb_factors_zz,
    fv_factors,
    fc_factors_yy, fc_factors_zz, fc_perp_factors,
    e_factors
    )

Checking for all sections:   0%|          | 0/1 [00:00<?, ?it/s]

..................................................
Calculando para la sección: sección 2
..................................................



Checking for all sections: 100%|██████████| 1/1 [00:00<?, ?it/s]

Calculando para la fuerza: force_1


In [23]:
section_result

,member,section,force,dcr_max,axial tension,axial compression,moment yy,moment zz,shear y,shear z,tension (dcr),biaxial bending (dcr),bending and tension (dcr),compression (dcr),bending and compression (dcr),shear y (dcr),shear z (dcr),compression perpendicular,compression perpendicular (dcr)
0,elemento 1,sección 2,force_1,0.000023,10000,0,0.0,0.0,10.0,10.0,0.002801,0.0,0.002801,0.0,0.0,0.000023,0.000023,10.0,0.533049


In [24]:
sections_results = design.check_for_all_sections(
    list_sections, list_members, list_forces, wood_material, ft_factors,
    fb_factors_yy, fb_factors_zz,
    fv_factors,
    fc_factors_yy, fc_factors_zz, fc_perp_factors,
    e_factors
    )

Checking for all sections:   0%|          | 0/3 [00:00<?, ?it/s]

..................................................
Calculando para la sección: sección 1
..................................................



Checking for all forces:   0%|          | 0/4 [00:00<?, ?it/s]


Calculando para la fuerza: force_1
Error processing section 'sección 1': 'list' object has no attribute 'name'
..................................................
Calculando para la sección: sección 2
..................................................



Checking for all forces:   0%|          | 0/4 [00:00<?, ?it/s]


Calculando para la fuerza: force_1
Error processing section 'sección 2': 'list' object has no attribute 'name'
..................................................
Calculando para la sección: sección 3
..................................................



Checking for all sections: 100%|██████████| 3/3 [00:00<00:00, 406.85it/s]

Calculando para la fuerza: force_1
Error processing section 'sección 3': 'list' object has no attribute 'name'

Errors encountered during processing:
Error processing section 'sección 1': 'list' object has no attribute 'name'
Error processing section 'sección 2': 'list' object has no attribute 'name'
Error processing section 'sección 3': 'list' object has no attribute 'name'


In [25]:
sections_results

""


In [26]:
element_results = design.check_for_all_elements(
    list_sections, element, list_forces, wood_material, ft_factors,
    fb_factors_yy, fb_factors_zz,
    fv_factors,
    fc_factors_yy, fc_factors_zz, fc_perp_factors,
    e_factors
    )

Checking for all sections:   0%|          | 0/1 [00:00<?, ?it/s]

::::::::::::::::::::::::::::::::::::::::::::::::::
Calculando para el elemento: elemento 1
::::::::::::::::::::::::::::::::::::::::::::::::::



cking for all sections:   0%|          | 0/3 [00:00<?, ?it/s]

..................................................
Calculando para la sección: sección 1
..................................................




Checking for all forces: 100%|██████████| 4/4 [00:00<?, ?it/s]


Calculando para la fuerza: force_1
Calculando para la fuerza: force_2
Calculando para la fuerza: force_3
Calculando para la fuerza: force_4
..................................................
Calculando para la sección: sección 2
..................................................




Checking for all forces: 100%|██████████| 4/4 [00:00<?, ?it/s]


Calculando para la fuerza: force_1
Calculando para la fuerza: force_2
Calculando para la fuerza: force_3
Calculando para la fuerza: force_4
..................................................
Calculando para la sección: sección 3
..................................................




Checking for all sections: 100%|██████████| 1/1 [00:00<00:00, 63.72it/s]

Calculando para la fuerza: force_1
Calculando para la fuerza: force_2
Calculando para la fuerza: force_3
Calculando para la fuerza: force_4


In [27]:
element_results

,member,section,force,dcr_max,axial tension,axial compression,moment yy,moment zz,shear y,shear z,tension (dcr),biaxial bending (dcr),bending and tension (dcr),compression (dcr),bending and compression (dcr),shear y (dcr),shear z (dcr),compression perpendicular,compression perpendicular (dcr)
0,elemento 1,sección 1,force_1,0.004582,10000,0.0,0.0,0.0,10.0,10.0,0.560224,0.000000,0.560224,0.000000,0.000000,0.004582,0.004582,10.0,0.533049
1,elemento 1,sección 1,force_2,0.012325,0,0.0,400.0,50.0,0.0,0.0,0.000000,0.012325,0.012325,0.000000,0.012325,0.000000,0.000000,0.0,0.000000
2,elemento 1,sección 1,force_3,0.595238,0,10000.0,1000.0,0.0,10.0,0.0,0.000000,0.022409,0.022409,0.595238,0.376717,0.004582,0.000000,0.0,0.000000
3,elemento 1,sección 1,force_4,0.119048,0,2000.0,400.0,50.0,10.0,20.0,0.000000,0.012325,0.012325,0.119048,0.026497,0.004582,0.009164,20.0,1.066098
4,elemento 1,sección 2,force_1,0.000023,10000,0.0,0.0,0.0,10.0,10.0,0.002801,0.000000,0.002801,0.000000,0.000000,0.000023,0.000023,10.0,0.533049
5,elemento 1,sección 2,force_2,0.000005,0,0.0,400.0,50.0,0.0,0.0,0.000000,0.000005,0.000005,0.000000,0.000005,0.000000,0.000000,0.0,0.000000
6,elemento 1,sección 2,force_3,0.002976,0,10000.0,1000.0,0.0,10.0,0.0,0.000000,0.000011,0.000011,0.002976,0.000020,0.000023,0.000000,0.0,0.000000
7,elemento 1,sección 2,force_4,0.000595,0,2000.0,400.0,50.0,10.0,20.0,0.000000,0.000005,0.000005,0.000595,0.000006,0.000023,0.000046,20.0,1.066098
8,elemento 1,sección 3,force_1,0.001527,10000,0.0,0.0,0.0,10.0,10.0,0.186741,0.000000,0.186741,0.000000,0.000000,0.001527,0.001527,10.0,0.533049
9,elemento 1,sección 3,force_2,0.003361,0,0.0,400.0,50.0,0.0,0.0,0.000000,0.003361,0.003361,0.000000,0.003361,0.000000,0.000000,0.0,0.000000


In [28]:
elements_results = design.check_for_all_elements(
    list_sections, list_members, list_forces, wood_material, ft_factors,
    fb_factors_yy, fb_factors_zz,
    fv_factors,
    fc_factors_yy, fc_factors_zz, fc_perp_factors,
    e_factors
    )

Checking for all sections:   0%|          | 0/3 [00:00<?, ?it/s]

::::::::::::::::::::::::::::::::::::::::::::::::::
Calculando para el elemento: elemento 1
::::::::::::::::::::::::::::::::::::::::::::::::::



cking for all sections:   0%|          | 0/3 [00:00<?, ?it/s]

..................................................
Calculando para la sección: sección 1
..................................................




Checking for all forces: 100%|██████████| 4/4 [00:00<?, ?it/s]


...

Calculando para la fuerza: force_1
Calculando para la fuerza: force_2
Calculando para la fuerza: force_3
Calculando para la fuerza: force_4
..................................................
Calculando para la sección: sección 2
..................................................




Checking for all forces: 100%|██████████| 4/4 [00:00<00:00, 3374.34it/s]


Calculando para la fuerza: force_1
Calculando para la fuerza: force_2
Calculando para la fuerza: force_3
Calculando para la fuerza: force_4
..................................................
Calculando para la sección: sección 3
..................................................




Checking for all sections: 100%|██████████| 3/3 [00:00<00:00, 63.37it/s]

Calculando para la fuerza: force_1
Calculando para la fuerza: force_2
Calculando para la fuerza: force_3
Calculando para la fuerza: force_4


In [29]:
elements_results

,member,section,force,dcr_max,axial tension,axial compression,moment yy,moment zz,shear y,shear z,tension (dcr),biaxial bending (dcr),bending and tension (dcr),compression (dcr),bending and compression (dcr),shear y (dcr),shear z (dcr),compression perpendicular,compression perpendicular (dcr)
0,elemento 1,sección 1,force_1,0.004582,10000,0.0,0.0,0.0,10.0,10.0,0.560224,0.000000,0.560224,0.000000,0.000000,0.004582,0.004582,10.0,0.533049
1,elemento 1,sección 1,force_2,0.012325,0,0.0,400.0,50.0,0.0,0.0,0.000000,0.012325,0.012325,0.000000,0.012325,0.000000,0.000000,0.0,0.000000
2,elemento 1,sección 1,force_3,0.595238,0,10000.0,1000.0,0.0,10.0,0.0,0.000000,0.022409,0.022409,0.595238,0.376717,0.004582,0.000000,0.0,0.000000
3,elemento 1,sección 1,force_4,0.119048,0,2000.0,400.0,50.0,10.0,20.0,0.000000,0.012325,0.012325,0.119048,0.026497,0.004582,0.009164,20.0,1.066098
4,elemento 1,sección 2,force_1,0.000023,10000,0.0,0.0,0.0,10.0,10.0,0.002801,0.000000,0.002801,0.000000,0.000000,0.000023,0.000023,10.0,0.533049
5,elemento 1,sección 2,force_2,0.000005,0,0.0,400.0,50.0,0.0,0.0,0.000000,0.000005,0.000005,0.000000,0.000005,0.000000,0.000000,0.0,0.000000
6,elemento 1,sección 2,force_3,0.002976,0,10000.0,1000.0,0.0,10.0,0.0,0.000000,0.000011,0.000011,0.002976,0.000020,0.000023,0.000000,0.0,0.000000
7,elemento 1,sección 2,force_4,0.000595,0,2000.0,400.0,50.0,10.0,20.0,0.000000,0.000005,0.000005,0.000595,0.000006,0.000023,0.000046,20.0,1.066098
8,elemento 1,sección 3,force_1,0.001527,10000,0.0,0.0,0.0,10.0,10.0,0.186741,0.000000,0.186741,0.000000,0.000000,0.001527,0.001527,10.0,0.533049
9,elemento 1,sección 3,force_2,0.003361,0,0.0,400.0,50.0,0.0,0.0,0.000000,0.003361,0.003361,0.000000,0.003361,0.000000,0.000000,0.0,0.000000


In [42]:
design.filter_and_export_results(
    results_df = element_results,
    filters = {
        'section': ['sección 1', 'sección 2'],
        'dcr_max': {'range': {'min': 0.00, 'max': 1.00}},
    },
    sort_by="dcr_max",
    sort_order = "desc"
)

Filtered results exported to: C:\Users\ingah\pycode\timber_nds\tests\test_data\filtered_results.xlsx


,member,section,force,dcr_max,axial tension,axial compression,moment yy,moment zz,shear y,shear z,tension (dcr),biaxial bending (dcr),bending and tension (dcr),compression (dcr),bending and compression (dcr),shear y (dcr),shear z (dcr),compression perpendicular,compression perpendicular (dcr)
2,elemento 1,sección 1,force_3,0.595238,0,10000.0,1000.0,0.0,10.0,0.0,0.000000,0.022409,0.022409,0.595238,0.376717,0.004582,0.000000,0.0,0.000000
10,elemento 1,sección 3,force_3,0.198413,0,10000.0,1000.0,0.0,10.0,0.0,0.000000,0.007470,0.007470,0.198413,0.046837,0.001527,0.000000,0.0,0.000000
3,elemento 1,sección 1,force_4,0.119048,0,2000.0,400.0,50.0,10.0,20.0,0.000000,0.012325,0.012325,0.119048,0.026497,0.004582,0.009164,20.0,1.066098
11,elemento 1,sección 3,force_4,0.039683,0,2000.0,400.0,50.0,10.0,20.0,0.000000,0.003361,0.003361,0.039683,0.004936,0.001527,0.003055,20.0,1.066098
1,elemento 1,sección 1,force_2,0.012325,0,0.0,400.0,50.0,0.0,0.0,0.000000,0.012325,0.012325,0.000000,0.012325,0.000000,0.000000,0.0,0.000000
0,elemento 1,sección 1,force_1,0.004582,10000,0.0,0.0,0.0,10.0,10.0,0.560224,0.000000,0.560224,0.000000,0.000000,0.004582,0.004582,10.0,0.533049
9,elemento 1,sección 3,force_2,0.003361,0,0.0,400.0,50.0,0.0,0.0,0.000000,0.003361,0.003361,0.000000,0.003361,0.000000,0.000000,0.0,0.000000
6,elemento 1,sección 2,force_3,0.002976,0,10000.0,1000.0,0.0,10.0,0.0,0.000000,0.000011,0.000011,0.002976,0.000020,0.000023,0.000000,0.0,0.000000
8,elemento 1,sección 3,force_1,0.001527,10000,0.0,0.0,0.0,10.0,10.0,0.186741,0.000000,0.186741,0.000000,0.000000,0.001527,0.001527,10.0,0.533049
7,elemento 1,sección 2,force_4,0.000595,0,2000.0,400.0,50.0,10.0,20.0,0.000000,0.000005,0.000005,0.000595,0.000006,0.000023,0.000046,20.0,1.066098
